In [1]:
import pandas as pd
from sqlalchemy import create_engine
import py
from config import gkey
import requests
import json
from pprint import pprint

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Yelp Data

### Yelp Transform

In [2]:
yelp_json = 'Resources/yelp_business.json'

#adding the commas to the json file
with open (yelp_json,'r', encoding='utf-8') as cmnts:
    all_entries = cmnts.readlines()
    
parsable_data = "[" + ','.join(all_entries).replace('\n','') + "]"
yelp_df = pd.read_json(parsable_data, encoding = 'utf-8')
yelp_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [3]:
# create dataframe with only the columns we want
new_yelp_df = yelp_df[['name','categories','address','city','state','postal_code','stars' ]].copy()
new_yelp_df.head()


,name,categories,address,city,state,postal_code,stars
0,Minhas Micro Brewery,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",1314 44 Avenue NE,Calgary,AB,T2E 6L6,4.0
1,CK'S BBQ & Catering,"Chicken Wings, Burgers, Caterers, Street Vendo...",,Henderson,NV,89002,4.5
2,La Bastringue,"Breakfast & Brunch, Restaurants, French, Sandw...",1335 rue Beaubien E,Montréal,QC,H2G 1K7,4.0
3,Geico Insurance,"Insurance, Financial Services",211 W Monroe St,Phoenix,AZ,85003,1.5
4,Action Engine,"Home & Garden, Nurseries & Gardening, Shopping...",2005 Alyth Place SE,Calgary,AB,T2H 0N5,2.0


In [11]:
# create dataframe for only Pittsburgh
pits_yelp_df = new_yelp_df.loc[new_yelp_df['city']=='Pittsburgh']
pits_yelp_df.head()


,name,categories,address,city,state,postal_code,stars
9,Maggie & Stella's Gifts,"Flowers & Gifts, Gift Shops, Shopping",209 Oakland Ave,Pittsburgh,PA,15213,3.5
67,Philip Pelusi,"Hair Salons, Beauty & Spas","531 Clairton Blvd, Unit 7",Pittsburgh,PA,15236,4.5
68,FedEx Office Print & Ship Center,"Professional Services, Local Services, Shippin...",1720 Washington Rd,Pittsburgh,PA,15241,3.5
73,China Palace,"Asian Fusion, Caterers, Fast Food, Chinese, Re...",5440 Walnut St,Pittsburgh,PA,15232,3.0
89,Pizza Bellagio,"Pizza, Restaurants",4635 Centre Ave,Pittsburgh,PA,15213,1.5


In [12]:
# limit businesses to only restaurants
pits_yelp_df = pits_yelp_df.dropna()
pits_yelp_df = pits_yelp_df[pits_yelp_df['categories'].str.contains('Restaurants')]
pits_yelp_df.head()


,name,categories,address,city,state,postal_code,stars
73,China Palace,"Asian Fusion, Caterers, Fast Food, Chinese, Re...",5440 Walnut St,Pittsburgh,PA,15232,3.0
89,Pizza Bellagio,"Pizza, Restaurants",4635 Centre Ave,Pittsburgh,PA,15213,1.5
295,Pittsburgh Poke,"Sushi Bars, Food, Thai, Poke, Japanese, Restau...",500 Liberty Ave,Pittsburgh,PA,15222,4.5
298,Schenley Park Visitors Center,"Cafes, Hotels & Travel, Swimming Pools, Travel...",101 Panther Hollow Rd,Pittsburgh,PA,15213,5.0
319,Crazy Mocha,"Food, Cafes, Coffee & Tea, Restaurants",1836 Centre Ave,Pittsburgh,PA,15219,5.0


In [13]:
#encode name column to UTF 8 (must run twice for export to work)
pits_yelp_df.name = pits_yelp_df.name.str.encode('utf-8')
pits_yelp_df.name = pits_yelp_df.name.str.encode('utf-8')

### Yelp Load

In [14]:
rds_connection_string = "root:HpFan10132@localhost/restaurant_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [15]:
#export to Mysql
pits_yelp_df.to_sql('yelp_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

# Yellow Pages Data

### Yellow Pages Transform

In [16]:
csv_file = "Resources/yellowpages.csv"

yellowpagesraw_df = pd.read_csv(csv_file)
yellowpagesraw_df.head()

,Uniq Id,Url,Name,Street,Zip Code,City,State,Phone,Email,Website,Categories,Rating
0,85c7c0ebada10ae36b0ffe6e72e92ed5,http://www.yellowpages.com/indianapolis-in/mip...,454,7280 Michigan Rd,46268.0,Indianapolis,IN,(317) 298-9001,NaN,NaN,"Continental Restaurants, Restaurants",1.0
1,e81c3082aafaf4e57883140616c516bd,http://www.yellowpages.com/philadelphia-pa/mip...,10 Arts,10 Avenue Of The Arts # Lobby,19102.0,Philadelphia,PA,(215) 523-8273,phila.10arts@ritzcarlton.com,http://www.10arts.com,"Continental Restaurants, French Restaurants, R...",1.5
2,92442b01e2f29fd1d8eef92ba2f91f09,http://www.yellowpages.com/miami-fl/mip/100-mo...,100 Montaditos,244 Biscayne Blvd,33132.0,Miami,FL,(786) 347-3065,NaN,http://www.100montaditos.com,"Spanish Restaurants, Restaurants, Sandwich Shops",2.0
3,f6d2b4e33a6c29860f34ef0217f33b97,http://www.yellowpages.com/indianapolis-in/mip...,120 West Market Fresh Grill,120 W Market St,46204.0,Indianapolis,IN,(317) 822-5824,NaN,NaN,Restaurants,2.5
4,0a78c2aa7744435ce55edde64c9254b6,http://www.yellowpages.com/philadelphia-pa/mip...,13th St Pizza,209 S 13th St,19107.0,Philadelphia,PA,(215) 546-4453,NaN,NaN,"Pizza, American Restaurants, Italian Restauran...",3.0


In [17]:
# create dataframe for only Pittsburgh
yellowpagespitt_df = yellowpagesraw_df.loc[yellowpagesraw_df["City"]=="Pittsburgh"]
yellowpagespitt_df.head()


,Uniq Id,Url,Name,Street,Zip Code,City,State,Phone,Email,Website,Categories,Rating
25,c0dea87ba09e8f82da541963b09a2b72,http://www.yellowpages.com/pittsburgh-pa/mip/5...,5 Fools,9566 Perry Hwy,15237.0,Pittsburgh,PA,(412) 366-3544,NaN,NaN,Restaurants,4.5
45,82f675b150359cf9b7153ba964a963f2,http://www.yellowpages.com/pittsburgh-pa/mip/a...,A&M Gyros,3536 Saw Mill Run Blvd,15227.0,Pittsburgh,PA,(412) 885-1010,NaN,NaN,"Greek Restaurants, Fast Food Restaurants, Medi...",1.0
162,b4cfe5fd4413d7fee6142bec935e8c80,http://www.yellowpages.com/pittsburgh-pa/mip/a...,Anthony's,950 Ohio River Blvd,15202.0,Pittsburgh,PA,(412) 734-4000,KillenR@astampcoin.com,http://anthonysfla.com,"Caterers, Restaurants",1.0
169,65bf7d01bb7fba7337e6efc13c84c4ff,http://www.yellowpages.com/pittsburgh-pa/mip/a...,Antonio's Pizzeria,758 Brookline Blvd,15226.0,Pittsburgh,PA,(412) 388-1000,NaN,http://antonios-pizza.com,"Pizza, Restaurants",4.5
448,0256d1b01ca051b457ab423a66cf8416,http://www.yellowpages.com/pittsburgh-pa/mip/b...,BLIMPIE,307 Smithfield St,15222.0,Pittsburgh,PA,(412) 281-0099,NaN,http://www.blimpie.com,"Delicatessens, Take Out Restaurants, Sandwich ...",4.5


In [18]:
# create dataframe with only the columns we want
yppitt_df = yellowpagespitt_df[["Name", "Street", "City","State", "Rating"]]
yppitt_df.head()


,Name,Street,City,State,Rating
25,5 Fools,9566 Perry Hwy,Pittsburgh,PA,4.5
45,A&M Gyros,3536 Saw Mill Run Blvd,Pittsburgh,PA,1.0
162,Anthony's,950 Ohio River Blvd,Pittsburgh,PA,1.0
169,Antonio's Pizzeria,758 Brookline Blvd,Pittsburgh,PA,4.5
448,BLIMPIE,307 Smithfield St,Pittsburgh,PA,4.5


In [19]:
# rename columns
YP_df = yppitt_df.rename(columns={"Name":"Restaurant_YP", "Street":"address", "City":"City_YP","State":"State_YP", "Rating":"Rating_YP"})
YP_df.head()

,Restaurant_YP,address,City_YP,State_YP,Rating_YP
25,5 Fools,9566 Perry Hwy,Pittsburgh,PA,4.5
45,A&M Gyros,3536 Saw Mill Run Blvd,Pittsburgh,PA,1.0
162,Anthony's,950 Ohio River Blvd,Pittsburgh,PA,1.0
169,Antonio's Pizzeria,758 Brookline Blvd,Pittsburgh,PA,4.5
448,BLIMPIE,307 Smithfield St,Pittsburgh,PA,4.5


### Yellow Pages Load

In [20]:
YP_df.to_sql('yp_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

# Google Data

### Google Extract

In [21]:
# Google Geocode API 
target_city = "Pittsburgh, Pennsylvania"

# set up a parameters dictionary
params = {"address": target_city, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Run request
response = requests.get(base_url, params=params)

# Convert to JSON
jc_geo = response.json()

# Extract lat/lng
lat = jc_geo["results"][0]["geometry"]["location"]["lat"]
lng = jc_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print(f"{target_city}: {lat}, {lng}")

Pittsburgh, Pennsylvania: 40.44062479999999, -79.9958864


In [22]:
# Google Places API 

# geocoordinates
target_coordinates = "40.44062479999999, -79.9958864"
target_radius = 1000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
pitt_data = response.json()

# Print the json (pretty printed)
print(json.dumps(pitt_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "CtQETgIAABCS3EcsmrmObII_azIO5IoL9TN0733IDhUpW8U8xdek_qFuVyWa1hrcBfpMZUc1ZXr4lwdu5BGHSlWs4tqVvrTdVpf0u9GAdAUaZL2Ja8E3Vb5KzUoTg1Pfo-rmCDL_Iaq2-3Mdbvu6L5B_o33eb_j_I70R9g-bASwyrrODpEjiOtxewLmHrBiZqknd1ieOWE-0c_tZOrHq_A3EeLkGloIWsCnsXFSeiRexFjiVv9Uh96Uewva_JdDXgR_KWa_04sqdQnIi9mF1vwNcG4KZ6IQ8ObzB7O8kDYGhWxOV_-aYSDJRK1cafeUCKA4l9nBF-yRGCceByWt_q4APyGtmsgAipCpKHped5vqAfpvRMPGyvKecNx_SPHBSGVc0dQz9rs0Nb1wncd9HmVGWt6sAbPBozj8w9kv3CaQnvsAThRJErMzl1OW4fND9t5c9FuNvWipIaFJjxTHw8SRntYpGoTGzTvug07vw_h3K2oEG6qONSUz5Ln6FhBl5FmOM_0bwtL-eRMG0AuhIMGCdKt122Pc1VS0bRcanBqQ29bKs08vJNeFvpWx6ZaQrCzD8mWR7R7D7K4yoGM5kS52NstOaVoWJJUMP-BzTLMyIGmXQBbI7VgHEhScaSvmQrA4KT0AvvAkqjyaOtRWmiFRsgZ8jSY57z5gG7ERdnuZU4kI5vVrBZquLJayNHyRWeyZ4WodLmCCY32siSuDIux_W0rRakT-xpiebanSbgu2D-YFt5Zr3VFyM7jWfFdXZi18O1I6jeloo2EfcV8lNvf75uC7gjC4SEAC_QNp94kAiswGscptqDhYaFFs7aoAV3CyS6UwScmw09f_0LMMV",
    "results": [
        {
            "geometry": {
                "location": {
       

In [23]:
# show only the "results"
pitt_list = pitt_data["results"]
pprint(pitt_list)


[{'geometry': {'location': {'lat': 40.44315030000001, 'lng': -80.0011895},
               'viewport': {'northeast': {'lat': 40.44443038029149,
                                          'lng': -79.9998085197085},
                            'southwest': {'lat': 40.44173241970849,
                                          'lng': -80.0025064802915}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
  'id': 'e4853e16b92352af2db90e7606628d66b541a725',
  'name': 'Meat & Potatoes',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 626,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/101632382784807640089/photos">Meat '
                                    '&amp; Potatoes</a>'],
              'photo_reference': 'CmRaAAAAymJzdVQ2IgtbDesZIGiMwE1cpPEaZscfZpw2ngAMYngsEOvpx_41VuKUxtk-D7Z6ylPJ4qJGwgA_AsxaVj1uxhXTJyfFCqUk2QzboLikll6sLBE_VEmaGhS9R-Bl09oAEhC6Zfc4g-mCJeCVqfAcnxolGhRdE6g

In [24]:
# lists to store JSON data
name = []
address = []
rating = []

# loop to append data to lists
for restaurant in pitt_list:
   name.append(restaurant["name"])
   address.append(restaurant["vicinity"])
   rating.append(restaurant["rating"])

df = pd.DataFrame({"name":name,"vicinity":address,"rating":rating})
df

,name,vicinity,rating
0,Meat & Potatoes,"649 Penn Avenue, Pittsburgh",4.5
1,McCormick & Schmick's Seafood & Steaks,"301 Fifth Avenue, Pittsburgh",4.0
2,Olive Or Twist,"140 6th Street, Pittsburgh",4.1
3,The Carlton Restaurant,"500 Grant Street, Pittsburgh",4.5
4,Bigelow Grille,"1 Bigelow Square, Pittsburgh",4.1
5,The Capital Grille,"301 Fifth Avenue, Pittsburgh",4.6
6,Steelhead Brasserie and Wine Bar,"112 Washington Place, Pittsburgh",3.3
7,Braddock's Rebellion,"107 6th Street, Pittsburgh",4.1
8,Primanti Bros.,"2 Market Square, Pittsburgh",4.3
9,Mexico City,"409 Wood Street, Pittsburgh",3.4


### Google Transform

In [25]:
# rename columns
LG_df = df.rename(columns={"name":"Restaurant_G", "vicinity":"Address_G", "rating":"Rating_G"})
LG_df


,Restaurant_G,Address_G,Rating_G
0,Meat & Potatoes,"649 Penn Avenue, Pittsburgh",4.5
1,McCormick & Schmick's Seafood & Steaks,"301 Fifth Avenue, Pittsburgh",4.0
2,Olive Or Twist,"140 6th Street, Pittsburgh",4.1
3,The Carlton Restaurant,"500 Grant Street, Pittsburgh",4.5
4,Bigelow Grille,"1 Bigelow Square, Pittsburgh",4.1
5,The Capital Grille,"301 Fifth Avenue, Pittsburgh",4.6
6,Steelhead Brasserie and Wine Bar,"112 Washington Place, Pittsburgh",3.3
7,Braddock's Rebellion,"107 6th Street, Pittsburgh",4.1
8,Primanti Bros.,"2 Market Square, Pittsburgh",4.3
9,Mexico City,"409 Wood Street, Pittsburgh",3.4


In [26]:
# new data frame with split value columns 
g_df = LG_df["Address_G"].str.split(", ", n = 2, expand = True) 
  
# making seperate address column from new data frame 
LG_df["Address_G"]= g_df[0] 

# add column into dataframe
LG_df = LG_df.rename(columns = {'Address_G': 'address'})
LG_df


,Restaurant_G,address,Rating_G
0,Meat & Potatoes,649 Penn Avenue,4.5
1,McCormick & Schmick's Seafood & Steaks,301 Fifth Avenue,4.0
2,Olive Or Twist,140 6th Street,4.1
3,The Carlton Restaurant,500 Grant Street,4.5
4,Bigelow Grille,1 Bigelow Square,4.1
5,The Capital Grille,301 Fifth Avenue,4.6
6,Steelhead Brasserie and Wine Bar,112 Washington Place,3.3
7,Braddock's Rebellion,107 6th Street,4.1
8,Primanti Bros.,2 Market Square,4.3
9,Mexico City,409 Wood Street,3.4


### Google Load

In [27]:
LG_df.to_sql('google_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)